<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-data-set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del Dataset

Durante este proceso se crea un dataset con lo datos procesados y listos para el entrenamiento del modelo.

# Parametros

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.
* dataset_path: ruta en el que se almacenará el dataset
* labels_file: ruta en la que se almacenará la lista de etiquetas

In [ ]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Datos de origen
ruta_archivo_GoldStandardTrainingEA = ruta_base + 'GoldStandardTrainingEAs.txt'
ruta_archivo_GoldStandardTestEA = ruta_base + 'GoldStandardTestEAs.txt'
ruta_archivos_entrenamiento = ruta_base + 'Training/'
ruta_archivos_test = ruta_base + 'Test/'

#Dataset procesado
ruta_dataset = ruta_base + 'Dataset'

In [ ]:
!pip install -q datasets unidecode nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.1 MB/s eta 0:00:00


Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


#Creación del dataset




Se crea un dataset con los tres segmentos: train, validation y test. Las etiquetas de train y validation se toman del fichero GoldStandardTrainingEAs.txt y las etiquetas del segmento test se toman del fichero GoldStandardTestEAs.txt.

Se cargan los archivos para su tratamiento.

In [ ]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    ruta_archivo_GoldStandardTrainingEA, sep=' ', header=None, names=column_names)
df_goldStandardTestEAs = pd.read_csv(
    ruta_archivo_GoldStandardTestEA, sep=' ', header=None, names=column_names)

Generación de las etiquetas

In [ ]:
df_goldStandardTrainingEAs

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27690409,T50.2X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24626366,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27967924,O90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25774939,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24878023,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14497,24368169,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14498,26850773,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,26850775,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14500,26850771,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from datasets import ClassLabel

def crear_lista_etiquetas(dataframe):
  label_list = list(dataframe.TieneEA_EA1.unique())
  label_list.extend(list(dataframe.EA2.unique()))
  label_list.extend(list(dataframe.EA3.unique()))
  label_list.extend(list(dataframe.EA4.unique()))
  label_list.extend(list(dataframe.EA5.unique()))
  label_list.extend(list(dataframe.EA6.unique()))
  label_list.extend(list(dataframe.EA7.unique()))
  label_list.extend(list(dataframe.EA8.unique()))
  label_list.extend(list(dataframe.EA9.unique()))
  label_list.extend(list(dataframe.EA10.unique()))
  df = pd.DataFrame(label_list, columns=["label"])
  df = df.dropna()
  label_list = list(df.label.unique())
  return ClassLabel(num_classes=len(label_list), names=label_list)

class2label_entrenamiento = crear_lista_etiquetas(df_goldStandardTrainingEAs)
class2label_test = crear_lista_etiquetas(df_goldStandardTestEAs)

Precesamiento del texto de los informes

In [ ]:
import json

str1 = "motivo de consulta"
str2 = "abadc add ad ad da da adsd"
str3 = "tratamiento"
str4 = "qwer qwrqwer eqwrqwer qwewqrq"

dictionary = {}

dictionary[str1] = str2
dictionary[str3] = str4

print(dictionary)

json_array = [ {'apartado' : k, 'texto' : dictionary[k]} for k in dictionary]
print(json.dumps(json_array))



{'motivo de consulta': 'abadc add ad ad da da adsd', 'tratamiento': 'qwer qwrqwer eqwrqwer qwewqrq'}
[{"apartado": "motivo de consulta", "texto": "abadc add ad ad da da adsd"}, {"apartado": "tratamiento", "texto": "qwer qwrqwer eqwrqwer qwewqrq"}]


In [ ]:
import os
import re
from unidecode import unidecode
from nltk.corpus import stopwords 
import nltk
import json

JUICIO_CLINICO = 'juicio clinico'
JUICIO_DIAGNOSTICO = 'juicio diagnostico'
TRATAMIENTO = 'Tratamiento'
FIRMADO_XXX = '_FIRMADO_XXXX_'
ETIQUETAS = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta", 
             "analitica", "comentarios", "juicio","clinico","tratamiento",
             "recomendaciones","exploraciones", "complementarias"]

re_sexo = re.compile(r'sexo [a-z]*')
re_juicio = re.compile(r'juicio [a-z]*')
re_gestacion = re.compile(r'gestacion [a-z]*')

campos = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta"]

apartados_de_interes = [
    "jdtco", "jc"] #, "comentarios"

apartados_descartados = [
    "recomendaciones", "lopd"]    

TRAINING = 0
TEST = 1

nltk.download('stopwords')

def omitir_textos_anonimizacion (linea):
  # busca expresiones del tipo _HOSPITAL_XXXX_ _DIRECCION_XXXX_
  return re.sub (r'_\w+_XXXX_', '', linea)

def omitir_no_texto (linea):
  return re.sub (r'[^\w]', ' ', linea)

def omitir_numeros (linea):
  return re.sub(r'[0-9]+', ' ', linea)

def es_linea_comienzo_seleccion (linea):
  return normalizar_texto(linea.strip()) == JUICIO_CLINICO

def es_linea_fin_seleccion (linea):
  return FIRMADO_XXX in linea.strip()

def linea_tiene_contenido (linea):
  return linea.strip() != ''

def es_apartado_interes (linea):
  for apartado in apartados_de_interes:
    if (apartado in linea):
      return True
  return False

def es_apartado (linea):
  return (re_juicio.match (linea) != None or
          re_gestacion.match (linea) != None or
          es_apartado_interes (linea))

def es_apartado_descartado (linea):
  for apartado in apartados_descartados:
    if (apartado in linea):
      return True
  return False

def detectar_etiquetas (linea):
  words = linea.strip().split()
  if len(words) >= 1 and len(words) <= 3:
      return linea
  return ""      

def omitir_stopwors (linea):
  stop_words = set(stopwords.words('spanish')) 
  palabras = linea.split() 
  resultado = ''
  for paralabra in palabras: 
      if not resultado in stop_words: 
        resultado += ' ' + paralabra
  return resultado

def normalizar_texto (linea):
  linea = unidecode(linea) 
  linea = linea.lower().strip()
  return linea

def quitar_etiquetas (linea):
  for etiqueta in ETIQUETAS:
    linea = linea.replace(etiqueta, "")
  return linea

def limpiar_texto (informe_medico, diccionario_apartados):
  texto = ''
  seleccionar = False  
  apartado = ""
  texto_apartado = ""

  for lineaTexto in informe_medico:
    lineaTexto = omitir_textos_anonimizacion(lineaTexto)
    lineaTexto = normalizar_texto(lineaTexto)
    lineaTexto = omitir_no_texto(lineaTexto)
    lineaTexto = omitir_numeros(lineaTexto)
    if (es_apartado(lineaTexto)):
      if (apartado != "" and apartado != lineaTexto):
        diccionario_apartados[apartado] = texto_apartado
        texto_apartado = ""
      apartado = lineaTexto   
      seleccionar = True   
    else:
      if (es_apartado_descartado(lineaTexto)):        
        if (texto_apartado != ""):
          diccionario_apartados[apartado] = texto_apartado
        apartado = "" 
        texto_apartado = ""
        seleccionar = False   
    if (seleccionar):      
      lineaTexto = omitir_stopwors(lineaTexto)  
      texto_apartado += lineaTexto 

  if (texto_apartado != ""):
   diccionario_apartados[apartado] = texto_apartado

  json_apartados = [ {'apartado' : k, 'texto' : diccionario_apartados[k]} for k in diccionario_apartados]
  return texto, json_apartados

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location, class2label):  
  texto_acto = ''  
  diccionario_apartados = {}
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico:
      resultado = limpiar_texto(informe_medico, diccionario_apartados)
      texto_acto = texto_acto + resultado[0]
      texto_acto = texto_acto + "\n"
  return [acto,
          class2label.str2int(label_list[0].strip()), 
          label_list[0].strip(), 
          label_list, 
          lista_nombres_informes_medicos, 
          texto_acto, 
          json.dumps(resultado[1])]

def tomar_codigos_EA_de_todas_las_columnas(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col])
  return label_list

def crear_vector_datos (dataframe, file_list, files_location, class2label): 
  dataframe_SinNone = dataframe.query("TieneEA_EA1 != 'NONE'")
  dataframe_SinNone = dataframe_SinNone.reset_index()  
  ds_resultado = []
  i = 0
  for index, row in dataframe_SinNone.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    ds_resultado.append(obtener_texto(row['Acto'], tomar_codigos_EA_de_todas_las_columnas(row), 
                                      list(archivos_acto), files_location, class2label))
    i += 1
    if (i > 20):
     break
  return ds_resultado

def crear_dataframe (labeled_array):
  return pd.DataFrame(labeled_array, 
                      columns=['acto', 'label', 'label_str', 'labels_str', 'informes', 'text', 'json'])

def generate_labeled_arrays_test ():
  training_files = ["27690409-171690541.txt"]
  labeled_array_training = crear_vector_datos (
      df_goldStandardTrainingEAs.query("Acto == 27690409"), training_files, 
        ruta_archivos_entrenamiento, class2label_entrenamiento)
  ds_labeled_training = crear_dataframe (labeled_array_training)
  return ds_labeled_training  

def crear_dataframe_train_test ():
  lista_archivos_entrenamiento = [f for f in os.listdir(ruta_archivos_entrenamiento) 
                      if os.path.isfile(os.path.join(ruta_archivos_entrenamiento, f))]
  vector_datos_entrenamiento = crear_vector_datos (
      df_goldStandardTrainingEAs, lista_archivos_entrenamiento, 
      ruta_archivos_entrenamiento, class2label_entrenamiento)
  df_entrenamiento = crear_dataframe (vector_datos_entrenamiento)
  
  lista_datos_test = [f for f in os.listdir(ruta_archivos_test) 
                  if os.path.isfile(os.path.join(ruta_archivos_test, f))]
  vector_datos_test = crear_vector_datos (
      df_goldStandardTestEAs, lista_datos_test, ruta_archivos_test, class2label_test)
  df_test = crear_dataframe (vector_datos_test)

  return df_entrenamiento, df_test



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = generate_labeled_arrays_test()

apartado:  lineaTexto: comentarios
apartado: comentarios lineaTexto: juicio clinico
apartado descartado: recomendaciones
apartado descartado: en cumplimiento del articulo   de la lopd     se informa que sus datos identificativos y de salud seran objeto de tratamiento e incorporados a los ficheros de datos sanitarios  cuya titularidad corresponde al     los datos unicamente seran utilizados con fines asociados a la atencion y gestion sanitaria  docencia y seguimiento asistencial  estando prevista su comunicacion a los organismos publicos con competencia en materia sanitaria 
[{"apartado": "comentarios", "texto": " comentarios ingresa por cuadro de insuficiencia cardiaca por lo que se inicia desde urgencias tratamiento depletivo con buena evolucion y balance negativo pero con deterioro secundario de la funcion renal por lo que finalmente se suspende diuretico se anade sueroterapia con mejoria parcial de la funcion renal pero persiste con leve deterioro del mismo asi como elevacion discre

In [ ]:
dataframes = crear_dataframe_train_test()

apartado:  lineaTexto: comentarios
apartado: comentarios lineaTexto: juicio clinico
apartado descartado: recomendaciones
apartado descartado: en cumplimiento del articulo   de la lopd     se informa que sus datos identificativos y de salud seran objeto de tratamiento e incorporados a los ficheros de datos sanitarios  cuya titularidad corresponde al     los datos unicamente seran utilizados con fines asociados a la atencion y gestion sanitaria  docencia y seguimiento asistencial  estando prevista su comunicacion a los organismos publicos con competencia en materia sanitaria 
[{"apartado": "comentarios", "texto": " comentarios ingresa por cuadro de insuficiencia cardiaca por lo que se inicia desde urgencias tratamiento depletivo con buena evolucion y balance negativo pero con deterioro secundario de la funcion renal por lo que finalmente se suspende diuretico se anade sueroterapia con mejoria parcial de la funcion renal pero persiste con leve deterioro del mismo asi como elevacion discre

In [ ]:
dataframes[0]

,acto,label,label_str,labels_str,informes,text,json
0,27690409,0,T50.2X5A,[T50.2X5A],[27690409-171690541.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."
1,27967924,2,O90.0,[O90.0],[27967924-172443607.txt],\n,"[{""apartado"": ""gestacion actual"", ""texto"": "" g..."
2,24626357,3,T46.5X5A,[T46.5X5A],[24626357-162859780.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."
3,24854067,4,E89.0,[E89.0],[24854067-163462099.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."
4,28005899,5,H59.022,[H59.022],[28005899-172516582.txt],\n,"[{""apartado"": ""juicio diagnostico"", ""texto"": ""..."
5,24878012,6,Y84.2,[Y84.2],[24878012-163509030.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."
6,24866017,0,T50.2X5A,[T50.2X5A],[24866017-163460301.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."
7,27448436,7,T83.021A,"[T83.021A, T83.511A, T83.091A, Y84.6]",[27448436-171066490.txt],\n,"[{""apartado"": ""juicio clinico "", ""texto"": "" ju..."
8,26518198,8,K94.23,[K94.23],[26518198-168286215.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."
9,24473926,9,T50.8X5A,[T50.8X5A],[24473926-162426171.txt],\n,"[{""apartado"": ""comentarios"", ""texto"": "" coment..."


Composición del dataset

In [ ]:
from datasets import Dataset, Value, DatasetDict

def set_dataset_features (dataset, class2label):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
dataset_entrenamiento = set_dataset_features (dataframes[TRAINING], class2label_entrenamiento)
dataset_test = set_dataset_features (dataframes[TEST], class2label_test)

dataset_entrenamiento_validacion = dataset_entrenamiento.train_test_split(test_size=0.1)

dataset_completo = DatasetDict(
    {
        "train": dataset_entrenamiento_validacion["train"],
        "validation": dataset_entrenamiento_validacion["test"],
        "test": dataset_test,
    })

dataset_completo

Casting the dataset:   0%|          | 0/2321 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/583 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 2088
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 233
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 583
    })
})

In [ ]:
dataset_completo["train"][0]

{'acto': 26557591,
 'label': 91,
 'label_str': 'T36.95XA',
 'labels_str': ['T36.95XA'],
 'informes': ['26557591-168440062.txt'],
 'text': "   \nSexo :Hombre\n\tINFORME ALTA DE MEDICINA INTERNA\n\tNIF/  \n\t  \n\t  \n\t  \n\tPaís: \n\tTeléfono:   \n\tTeléfono Móvil:   \n\tFecha ingreso:      \nFecha alta:         \n\t  \n\tTipo de Ingreso: Desde urgencias\n\tMotivo de Ingreso (MAM) hepatitis aguda colestasica toxica\nMotivo de Alta Mejoría\nMotivo de consulta dolor abdominal e ictericia\n  \n** ANTECEDENTES PERSONALES: \n· No reacciones alérgicas medicamentosas conocidas. \n· No hábitos tóxicos (tabaco, alcohol, drogas…). \n· Factores de riesgo cardiovascular Hipertensión arterial Y Dislipemia y NO Diabetes Mellitus,\n· Cardiopatía isquémica crónica: ingreso en 2004 por angina inestable (angina de mínimos esfuerzos de 1 mes de evolución, con isquemia subepicárdica anterior en ECG). DA ocluida en segmento medio, tratada con ACTP-stent recubierto con buen resultado angiográfico. ETT con F

In [ ]:
dataset_completo["train"][0]['text']

"   \nSexo :Hombre\n\tINFORME ALTA DE MEDICINA INTERNA\n\tNIF/  \n\t  \n\t  \n\t  \n\tPaís: \n\tTeléfono:   \n\tTeléfono Móvil:   \n\tFecha ingreso:      \nFecha alta:         \n\t  \n\tTipo de Ingreso: Desde urgencias\n\tMotivo de Ingreso (MAM) hepatitis aguda colestasica toxica\nMotivo de Alta Mejoría\nMotivo de consulta dolor abdominal e ictericia\n  \n** ANTECEDENTES PERSONALES: \n· No reacciones alérgicas medicamentosas conocidas. \n· No hábitos tóxicos (tabaco, alcohol, drogas…). \n· Factores de riesgo cardiovascular Hipertensión arterial Y Dislipemia y NO Diabetes Mellitus,\n· Cardiopatía isquémica crónica: ingreso en 2004 por angina inestable (angina de mínimos esfuerzos de 1 mes de evolución, con isquemia subepicárdica anterior en ECG). DA ocluida en segmento medio, tratada con ACTP-stent recubierto con buen resultado angiográfico. ETT con FEVI conservada, IM leve. Rev. en Cardiología hasta 2009, ergometría en 2005, 2006 y 2009 negativas. \n· En Urología desde 2013 por elevaci

In [ ]:
dataset_completo["train"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['T50.2X5A', 'NONE', 'O90.0', 'T46.5X5A', 'E89.0', 'H59.022', 'Y84.2', 'T83.021A', 'K94.23', 'T50.8X5A', 'Y95', 'T50.905A', 'T50.0X5A', 'L27.0', 'T85.398A', 'T38.0X5A', 'T85.79XA', 'P39.1', 'T84.498A', 'T84.82XA', 'K91.841', 'I97.618', 'T80.1XXA', 'T84.89XA', 'T40.605A', 'R50.82', 'T81.4XXA', 'T38.3X5A', 'G97.1', 'O75.2', 'N99.820', 'T45.1X5A', 'T39.1X5A', 'T85.71XA', 'L76.02', 'K66.0', 'Y83.1', 'T82.120A', 'K12.31', 'P01.1', 'G25.1', 'G89.18', 'T38.0X5D', 'L76.31', 'M96.830', 'G97.41', 'T47.4X5A', 'N99.821', 'T45.515A', 'T80.211A', 'T46.2X5S', 'P36.9', 'T36.0X5A', 'T84.84XA', 'G62.0', 'D70.1', 'T82.868A', 'T79.6XXA', 'T40.2X5A', 'T39.395A', 'T41.5X5A', 'T84.223A', 'T45.1X5D', 'T83.498A', 'L76.22', 'T43.295A', 'K94.12', 'P39.8', 'O86.29', 'K91.71', 'T84.021A', 'M96.840', 'Y83.2', 'T87.81', 'I97.130', 'T46.0X5A', 'T84.51XA', 'P03.4', 'Y64.0', 'J95.811', 'T39.2X5A', 'G97.51', 'T48.6X5A', 'T45.615A', 'H59.021', 'T82.7XXA',

In [ ]:
dataset_completo["test"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['NONE', 'T38.0X5A', 'Y83.1', 'P02.69', 'T45.515A', 'T36.95XA', 'T50.2X5A', 'T39.315A', 'T39.95XA', 'O91.22', 'T86.10', 'Y95', 'T39.395A', 'T84.54XA', 'T84.226A', 'P01.1', 'K91.840', 'J95.811', 'T82.7XXA', 'T43.505A', 'T50.8X5A', 'E89.0', 'T45.1X5A', 'K94.29', 'T81.83XA', 'T45.615A', 'T86.5', 'K91.841', 'T88.8XXA', 'T80.1XXA', 'L76.22', 'O75.2', 'T46.0X5A', 'I97.190', 'O04.5', 'T81.4XXA', 'I95.2', 'T88.59XA', 'T36.1X5A', 'M96.1', 'T42.8X5A', 'O75.82', 'I97.121', 'T83.51XA', 'Y83.6', 'K66.0', 'T46.2X5A', 'Y84.2', 'L76.32', 'P03.89', 'K91.71', 'T84.428A', 'T85.49XA', 'T50.905A', 'G72.0', 'T83.89XA', 'T86.11', 'T42.4X5A', 'P03.4', 'K94.09', 'T80.212A', 'T39.2X5A', 'Y83.8', 'T39.4X5A', 'K12.31', 'P39.3', 'T82.09XD', 'K91.89', 'T82.9XXA', 'T82.868A', 'T42.75XA', 'G89.18', 'E89.89', 'R50.82', 'T50.3X5A', 'T40.2X5A', 'T82.524A', 'P13.4', 'T83.59XA', 'P02.7', 'T82.855D', 'T50.1X5A', 'P15.8', 'I97.51', 'T45.525A', 'N99.111', 'K9

In [ ]:
dataset_completo.save_to_disk(ruta_dataset)

Saving the dataset (0/1 shards):   0%|          | 0/2088 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/233 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/583 [00:00<?, ? examples/s]